### Importing necessary libraries

In [1]:
!pip install -r requirements.txt

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.


In [2]:
import cv2
import numpy as np
from random import randint

### Defining Object tracking algorithms for Multi object tracking

In [3]:
def create_tracker(trackertype):
    trackers_dict = {'BOOSTING' : cv2.TrackerBoosting_create(),
                 'MIL' : cv2.TrackerMIL_create(), 
                 'KCF' : cv2.TrackerKCF_create(),
                 'TLD' : cv2.TrackerTLD_create(), 
                 'MEDIANFLOW' : cv2.TrackerMedianFlow_create(), 
                 'GOTURN' : cv2.TrackerGOTURN_create(), 
                 'MOSSE' : cv2.TrackerMOSSE_create(), 
                 'CSRT' : cv2.TrackerCSRT_create()}
    if trackertype in trackers_dict.keys():
        tracker = trackers_dict[trackertype]
        
    else :
        tracker = None
        print('Incorrect tracker name')
        print('Available trackers :')
        for key in trackers_dict.keys():
            print(key)
    return tracker

### Importing the Face detector and eye dector from haarcascades
* Haarcascades download link reference : https://github.com/opencv/opencv/tree/master/data/haarcascades

In [4]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml') 

### Helper Functions to detect the pupil of eye

In [17]:
detector_params = cv2.SimpleBlobDetector_Params()
detector_params.filterByArea = True

def eyes_blob(face , threshold):
    grey_face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    grey_face = cv2.medianBlur(grey_face, 5)
    eyes = eye_cascade.detectMultiScale(grey_face)
    for (x1,y1,w1,h1) in eyes:
        area = w1*h1
        detector_params.minArea = area/12
        detector_params.maxArea =  area/5
        detector = cv2.SimpleBlobDetector_create(detector_params)
        cv2.rectangle(face,(x1,y1),(x1+w1,y1+h1),(0,255,0),2)
        eye = face[int(y1):int(y1+h1), int(x1):int(x1+w1)]
        grey_eye = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
        ret, grey_eye = cv2.threshold(grey_eye, threshold, 255, cv2.THRESH_BINARY)
        #grey_eye = cv2.GaussianBlur(grey_eye,(5,5),0)
        grey_eye = cv2.erode(grey_eye, None, iterations=2) #1
        #grey_eye = cv2.dilate(grey_eye, None, iterations=4) #2
        grey_eye = cv2.medianBlur(grey_eye, 5)
        keypoints = detector.detect(grey_eye)
        if keypoints :
            print(threshold,area, keypoints[0].pt[0],keypoints[0].pt[1])
        cv2.drawKeypoints(eye, keypoints, eye, (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

def get_blobs(eye , threshold, area):
    detector_params.minArea = area/12
    detector_params.maxArea =  area/5
    detector = cv2.SimpleBlobDetector_create(detector_params)
    grey_eye = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
    ret, grey_eye = cv2.threshold(grey_eye, threshold, 255, cv2.THRESH_BINARY)
    grey_eye = cv2.medianBlur(grey_eye, 5)
    grey_eye = cv2.erode(grey_eye, None, iterations=2) #1
    #grey_eye = cv2.dilate(grey_eye, None, iterations=4)
    keypoints = detector.detect(grey_eye)
    if keypoints :
        #print(threshold,area, keypoints[0].pt[0],keypoints[0].pt[1], keypoints[0].size)
        key_x,key_y,key_dia = keypoints[0].pt[0],keypoints[0].pt[1], keypoints[0].size
        cv2.drawKeypoints(eye, keypoints, eye, (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        return key_x,key_y,key_dia
    
def get_blob_track():
        xl,yl,wl,hl = left_eye
        l_eye = img[yl:yl+hl, xl:xl+wl]
        area = wl*hl
        t = 5
        while (True):
            a = get_blobs(l_eye , t, area)
            if a is not None:
                #print(a)
                key_x,key_y,l_dia = a
                #bbox = (int(xl+key_x-l_dia/2),int(yl+key_y-l_dia/2),int(l_dia),int(l_dia))
                #multiTracker.add(createTrackerByName(trackerType), img, bbox)
                break
            t += 1
        t1 = t
        xl,yl,wl,hl = right_eye
        r_eye = img[yl:yl+hl, xl:xl+wl]
        area = wl*hl
        t = 5
        while (True):
            a = get_blobs(r_eye , t, area)
            if a is not None:
                key_x,key_y,r_dia = a
                #bbox = (int(xl+key_x-r_dia/2),int(yl+key_y-r_dia/2),int(r_dia),int(r_dia))
                #multiTracker.add(createTrackerByName(trackerType), img, bbox)
                break
            t += 1
        t2 = t
        return [t1,t2]

### Face and eye detection method
**Pupil detection method :** Blobs are detected on the eyes detected from the detected faces. 

**Advantages:**
* Faces and eyes are detected in every frame so it doesn't loose track of the eyes and faces.
* This method works very accurately for high resolution webcams/ camera.
* starts detecting any new faces in the frame

**Limitations:**
* For low resolution cameras, the discontinuity in face and eye detection can be identified.
* If the eye detection accuracy is low the pupil detection becomes extremely difficult
* Threshold value varies for different lighting conditions, this should be set manually


**Please note that threshold should be set manually using the trackerbar in this method**

In [18]:
import cv2
def nothing(x):
    pass
cap = cv2.VideoCapture(0)
cv2.namedWindow('image')
cv2.createTrackbar('threshold', 'image', 0, 255, nothing)
while(True):
    threshold = cv2.getTrackbarPos('threshold', 'image')
    ret, img = cap.read()
    gray_picture = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_picture = cv2.medianBlur(gray_picture, 7)
    faces = face_cascade.detectMultiScale(gray_picture, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)
        face_left = img[int(y):int(y+h//2), int(x):int(x+w//2)]
        face_right = img[int(y):int(y+h//2), int(x+w//2):int(x+w)]
        eyes_blob(face_left , threshold)
        eyes_blob(face_right , threshold)
    cv2.imshow("image", img)
    c = cv2.waitKey(1)
    if c == 27:
        cv2.destroyAllWindows()
        break
cap.release()

28 1681 12.89159870147705 22.976152420043945
28 1600 16.733333587646484 22.700000762939453
28 1681 15.490078926086426 25.83333396911621
29 1521 16.42831802368164 22.31563377380371
29 961 8.057971000671387 19.004140853881836
29 1681 16.473039627075195 22.476102828979492
27 1225 10.762300491333008 20.466388702392578
32 1764 16.85236358642578 23.24336814880371
26 1444 8.853994369506836 22.891183853149414
26 1444 12.095041275024414 22.479339599609375


### Manually selecting Eye regions for detection

**Pupil detection method :** Blobs are detected on the selected eye tracking regions. 

**Advantages:**
* Works very fast as we are not detecting on each frame (Speed depends on the type of tracker which is being used
* Works even on the low resolution cameras.

**Limitations:**
* Eye regions should be selected manually
* The tracking of the pupil completely depends on the region of selection accuracy
* Threshold should be set manually

**Please note that threshold should be set manually using the trackerbar in this method**

In [22]:
import cv2
def nothing(x):
    pass
trackerType = 'CSRT'
multiTracker = cv2.MultiTracker_create()
cap = cv2.VideoCapture(0)
cv2.namedWindow('image')
k = 0
colors = [(0,255,0),(0,255,0)]
bboxes = []
while(True):
    ret, img = cap.read()
    if k == 0:
        cv2.imshow("image", img)
        print('Select the eye bounding boxes')
        while(True):
            bbox = cv2.selectROI('image', img)
            bboxes.append(bbox)
            colors.append((randint(64, 255), randint(64, 255), randint(64, 255)))
            m = cv2.waitKey(0) & 0xFF
            if len(bboxes) == 2:
                break
            else:
                print("Press any key to select next object")
        multiTracker = cv2.MultiTracker_create()

        # Initialize MultiTracker 
        #print(len(bbox))
        for bbox in bboxes:
            print(bbox)
            multiTracker.add(create_tracker(trackerType), img, bbox)
        k=1
        cv2.createTrackbar('threshold', 'image', 0, 255, nothing)
    threshold = cv2.getTrackbarPos('threshold', 'image')
    (success, boxes) = multiTracker.update(img)
    
    #print(len(boxes))
    #success1,face_bbox = tracker1.update(img)
    if not success:
        print('Failed')
    if success:
        for i, newbox in enumerate(boxes):
            #print(i,newbox)
            xi,yi,wi,hi = int(newbox[0]),int(newbox[1]),int(newbox[2]),int(newbox[3])
            cv2.rectangle(img, (xi,yi),(xi+wi,yi+hi), colors[i], 2, 1)
            if i > 0 :
                eye = img[yi:yi+hi, xi:xi+wi]
                area = wi*hi
                get_blobs(eye , threshold, area)
            
    cv2.imshow("image", img)
    c = cv2.waitKey(1)
    if c == 27:
        cv2.destroyAllWindows()
        break
cap.release()

Select the eye bounding boxes
Press any key to select next object
(262, 176, 68, 43)
(365, 186, 43, 40)


### Face and Eye detection and Tracking
Face and eye detection **Haarcascades** are used to detect face and eyes

**Pupil detection method :** Blobs are detected on the selected eye tracking regions. 

**Advantages:**
* Works very fast as we are not detecting on each frame (Speed depends on the type of tracker which is being used)
* Works even on the low resolution cameras.

**Limitations:**
* Threshold should be set manually
* Sometimes same threshold value doesnt work for both the eye regions

**Please note that threshold should be set manually using the trackerbar in this method**

In [20]:
import cv2
def nothing(x):
    pass
# CSRT tracker is used as the accuracy is higher, KCF tracker can be used for faster detection
trackerType = 'CSRT'
cap = cv2.VideoCapture(0)
cv2.namedWindow('image')
k = 0
s = 0
threshold = 0
colors = [(255,255,0),(0,255,0),(0,255,0),(0,0,255),(0,0,255)]
cv2.createTrackbar('threshold', 'image', 0, 255, nothing)
while(True):
    threshold = cv2.getTrackbarPos('threshold', 'image')
    while k < 4:
        ret, img = cap.read()
        cv2.imshow("image", img)
        c = cv2.waitKey(1)
        k = 0
        multiTracker = cv2.MultiTracker_create()
        bboxes = []
        ret, img = cap.read()
        gray_picture = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray_picture = cv2.medianBlur(gray_picture, 7)
        faces = face_cascade.detectMultiScale(gray_picture, 1.3, 5)
        if len(faces) > 0:
            face_bbox = (faces[0][0],faces[0][1],faces[0][2],faces[0][3])
            (x,y,w,h) = face_bbox
            bboxes.append(face_bbox)
            k+=1
        
        face_left = img[int(y):int(y+h//2), int(x):int(x+w//2)]
        grey_face = cv2.cvtColor(face_left, cv2.COLOR_BGR2GRAY)
        grey_face = cv2.medianBlur(grey_face, 5)
        eyes = eye_cascade.detectMultiScale(grey_face)
        if len(eyes) > 0:
            left_eye = (eyes[0][0]+x,eyes[0][1]+y,eyes[0][2],eyes[0][3])
            bboxes.append(left_eye)
            k+=1
        
        face_right = img[int(y):int(y+h//2), int(x+w//2):int(x+w)]
        grey_face = cv2.cvtColor(face_right, cv2.COLOR_BGR2GRAY)
        grey_face = cv2.medianBlur(grey_face, 5)
        eyes = eye_cascade.detectMultiScale(grey_face)
        if len(eyes) > 0:
            right_eye = (eyes[0][0]+x+w//2,eyes[0][1]+y,eyes[0][2],eyes[0][3])
            bboxes.append(right_eye)
            k+=1
        
        if k == 3:
            for bbox in bboxes:
                print(bbox)
                multiTracker.add(create_tracker(trackerType), img, bbox)
            #thr = get_blob_track()
            k+=1
        print(k)
        
    ret, img = cap.read()
    success, boxes = multiTracker.update(img)
    
    #print(len(boxes))
    #success1,face_bbox = tracker1.update(img)
    if not success:
        s += 1
        if s >= 30:
            k = 0
    if success:
        s = 0
        for i, newbox in enumerate(boxes):
            xi,yi,wi,hi = int(newbox[0]),int(newbox[1]),int(newbox[2]),int(newbox[3])
            cv2.rectangle(img, (xi,yi),(xi+wi,yi+hi), colors[i], 2, 1)
            if i > 0 :
                eye = img[yi+hi//10:yi+hi, xi:xi+wi]
                area = wi*hi
                try:
                    get_blobs(eye , threshold, area)
                except:
                    next
        
        #x,y,w,h = int(face_bbox[0]),int(face_bbox[1]),int(face_bbox[2]),int(face_bbox[3])
        #cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)
    cv2.imshow("image", img)
    c = cv2.waitKey(1)
    if c == 27:
        cv2.destroyAllWindows()
        break
cap.release()

0
0
0
0
0
(218, 94, 196, 196)
(245, 136, 54, 54)
(333, 138, 50, 50)
4


### Face and Eye detection and Tracking - Adaptive thresholding
This method is similar to the previous method, except that threshold is calculated automatically.

**Pupil detection method :** Blobs are detected on the selected eye tracking regions. 

**Advantages:**
* Works very fast as we are not detecting on each frame (Speed depends on the type of tracker which is being used)
* Works even on the low resolution cameras.
* Threshold values will be calculated automatically for each eye

**Limitations:**
* Fast movement of face might cause in losing the tracking

**Please note that threshold should be set manually using the trackerbar in this method**

In [32]:
import cv2
def nothing(x):
    pass
trackerType = 'CSRT'
cap = cv2.VideoCapture(0)
k = 0
s = 0
threshold = 0
colors = [(255,255,0),(0,255,0),(0,255,0),(0,0,255),(0,0,255)]
while(True):
    #threshold = cv2.getTrackbarPos('threshold', 'image')
    while k < 4:
        ret, img = cap.read()
        cv2.imshow("image", img)
        c = cv2.waitKey(1)
        k = 0
        multiTracker = cv2.MultiTracker_create()
        bboxes = []
        gray_picture = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray_picture = cv2.medianBlur(gray_picture, 7)
        faces = face_cascade.detectMultiScale(gray_picture, 1.3, 5)
        if len(faces) > 0:
            face_bbox = (faces[0][0],faces[0][1],faces[0][2],faces[0][3])
            (x,y,w,h) = face_bbox
            bboxes.append(face_bbox)
            k+=1
        
        face_left = img[int(y):int(y+h//2), int(x):int(x+w//2)]
        grey_face = cv2.cvtColor(face_left, cv2.COLOR_BGR2GRAY)
        grey_face = cv2.medianBlur(grey_face, 5)
        eyes = eye_cascade.detectMultiScale(grey_face)
        if len(eyes) > 0:
            left_eye = (eyes[0][0]+x,eyes[0][1]+y,eyes[0][2],eyes[0][3])
            bboxes.append(left_eye)
            k+=1
        
        face_right = img[int(y):int(y+h//2), int(x+w//2):int(x+w)]
        grey_face = cv2.cvtColor(face_right, cv2.COLOR_BGR2GRAY)
        grey_face = cv2.medianBlur(grey_face, 5)
        eyes = eye_cascade.detectMultiScale(grey_face)
        if len(eyes) > 0:
            right_eye = (eyes[0][0]+x+w//2,eyes[0][1]+y,eyes[0][2],eyes[0][3])
            bboxes.append(right_eye)
            k+=1
        
        if k == 3:
            for bbox in bboxes:
                print(bbox)
                multiTracker.add(create_tracker(trackerType), img, bbox)
            thr = get_blob_track()
            print(thr)
            k+=1
        print(k)
        
    ret, img = cap.read()
    success, boxes = multiTracker.update(img)
    if not success:
        s += 1
        if s >= 30:
            k = 0
    if success:
        s = 0
        for i, newbox in enumerate(boxes):
            xi,yi,wi,hi = int(newbox[0]),int(newbox[1]),int(newbox[2]),int(newbox[3])
            cv2.rectangle(img, (xi,yi),(xi+wi,yi+hi), colors[i], 2, 1)
            if i > 0 :
                eye = img[yi+hi//10:yi+hi, xi:xi+wi]
                area = wi*hi
                try:
                    get_blobs(eye , thr[i-1]+3, area)
                except:
                    next
    cv2.imshow("image", img)
    c = cv2.waitKey(1)
    if c == 27:
        cv2.destroyAllWindows()
        break
cap.release()

(207, 21, 223, 223)
(252, 82, 46, 46)
(342, 84, 42, 42)
[7, 9]
4


In [29]:
cv2.destroyAllWindows()
cap.release()

**References :**
1. https://medium.com/@stepanfilonov/tracking-your-eyes-with-python-3952e66194a6
2. https://www.learnopencv.com/multitracker-multiple-object-tracking-using-opencv-c-python/